In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux
numpy 1.16.3
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


# 2. Load Center Coords by Candidates

In [4]:
lungs = list(set(df_meta_test.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 222


In [5]:
list_tasks = []
size = settings.CUBE_SIZE
step = 32
start = [ size//2, size//2, size//2 ] # z,y,x

for uid in tqdm(lungs):
    meta = df_meta_test.loc[uid] # slice
    
    for z in range(start[0], int(meta.slice), step):
        for y in range(start[1], int(meta.height), step):
            for x in range(start[2], int(meta.width), step):
                task = {}
                task['seriesuid'] = uid
                task['vcoordX'] = x
                task['vcoordY'] = y
                task['vcoordZ'] = z
                
                # print(task)
                list_tasks.append(task)

100%|██████████| 222/222 [00:00<00:00, 251.08it/s]


In [6]:
df_tasks = pd.DataFrame(list_tasks, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
df_tasks = df_tasks.set_index('seriesuid')
df_tasks.index = df_tasks.index.astype('str')

print('total:', df_tasks.shape, 'lung:', len(set(df_tasks.index)))

if not os.path.exists(settings.SUBMISSION_DIR + 'candidates'):
    os.mkdir(settings.SUBMISSION_DIR + 'candidates')
    
df_tasks.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_cubes.csv', encoding='utf-8')


total: (208898, 3) lung: 222


# 3. Predict Results by tasks_cubes.csv

In [7]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '1'
output_dir = './output/unet3d/1564381015/' # 1. 1564381015 
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

W0729 12:01:06.616174 140037969671936 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 12:01:06.617924 140037969671936 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 12:01:06.624817 140037969671936 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [9]:
batch_size = 64
threshold_probability = 0.8
wtype = 'lung'
plot = False

In [10]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [11]:
list_results = []
print(wtype + 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_test.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='test', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred[:,:,:,0], itm)
                    if len(bboxes) > 0:
                        for i, box in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            if plot:
                                plot_cube((pred*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label +'.csv', encoding='utf-8')

    print('-'*100)
    

lungwindow
lung: 678189 tasks: 1100 meta: 360.0 350.0 350.0
647/647 [==============================] - 35s 54ms/sample
total: (35, 6) lung: 1
----------------------------------------------------------------------------------------------------
lung: 693095 tasks: 900 meta: 300.0 350.0 350.0
598/598 [==============================] - 10s 17ms/sample
total: (51, 6) lung: 2
----------------------------------------------------------------------------------------------------
lung: 680237 tasks: 900 meta: 300.0 350.0 350.0
610/610 [==============================] - 10s 17ms/sample
total: (73, 6) lung: 3
----------------------------------------------------------------------------------------------------
lung: 679953 tasks: 1089 meta: 290.0 381.0 381.0
552/552 [==============================] - 10s 18ms/sample
total: (83, 6) lung: 4
----------------------------------------------------------------------------------------------------
lung: 663709 tasks: 968 meta: 280.0 379.0 379.0
521/521 [======

518/518 [==============================] - 9s 17ms/sample
total: (775, 6) lung: 36
----------------------------------------------------------------------------------------------------
lung: 679968 tasks: 900 meta: 290.0 350.0 350.0
609/609 [==============================] - 10s 17ms/sample
total: (796, 6) lung: 37
----------------------------------------------------------------------------------------------------
lung: 694420 tasks: 1089 meta: 315.0 371.0 371.0
509/509 [==============================] - 8s 16ms/sample
total: (815, 6) lung: 38
----------------------------------------------------------------------------------------------------
lung: 688920 tasks: 1000 meta: 330.0 350.0 350.0
677/677 [==============================] - 12s 17ms/sample
total: (859, 6) lung: 39
----------------------------------------------------------------------------------------------------
lung: 647552 tasks: 900 meta: 305.0 324.0 324.0
587/587 [==============================] - 10s 16ms/sample
total: (8

684/684 [==============================] - 12s 17ms/sample
total: (2142, 6) lung: 106
----------------------------------------------------------------------------------------------------
lung: 631315 tasks: 1331 meta: 365.0 383.0 383.0
853/853 [==============================] - 14s 17ms/sample
total: (2163, 6) lung: 107
----------------------------------------------------------------------------------------------------
lung: 641942 tasks: 900 meta: 315.0 350.0 350.0
646/646 [==============================] - 10s 16ms/sample
total: (2177, 6) lung: 108
----------------------------------------------------------------------------------------------------
lung: 672198 tasks: 810 meta: 345.0 319.0 319.0
592/592 [==============================] - 9s 16ms/sample
total: (2189, 6) lung: 109
----------------------------------------------------------------------------------------------------
lung: 660058 tasks: 1210 meta: 345.0 369.0 369.0
693/693 [==============================] - 11s 16ms/sample


699/699 [==============================] - 11s 16ms/sample
total: (2707, 6) lung: 141
----------------------------------------------------------------------------------------------------
lung: 678534 tasks: 900 meta: 300.0 351.0 351.0
647/647 [==============================] - 10s 16ms/sample
total: (2726, 6) lung: 142
----------------------------------------------------------------------------------------------------
lung: 631062 tasks: 900 meta: 300.0 350.0 350.0
571/571 [==============================] - 9s 16ms/sample
total: (2734, 6) lung: 143
----------------------------------------------------------------------------------------------------
lung: 542609 tasks: 1521 meta: 315.0 439.0 439.0
596/596 [==============================] - 9s 16ms/sample
total: (2761, 6) lung: 144
----------------------------------------------------------------------------------------------------
lung: 672182 tasks: 1440 meta: 350.0 403.0 403.0
659/659 [==============================] - 10s 16ms/sample
t

368/368 [==============================] - 6s 16ms/sample
total: (3263, 6) lung: 176
----------------------------------------------------------------------------------------------------
lung: 661614 tasks: 1440 meta: 330.0 409.0 409.0
759/759 [==============================] - 12s 16ms/sample
total: (3292, 6) lung: 177
----------------------------------------------------------------------------------------------------
lung: 679971 tasks: 900 meta: 300.0 350.0 350.0
587/587 [==============================] - 9s 16ms/sample
total: (3341, 6) lung: 178
----------------------------------------------------------------------------------------------------
lung: 320473 tasks: 700 meta: 240.0 350.0 350.0
456/456 [==============================] - 7s 16ms/sample
total: (3375, 6) lung: 179
----------------------------------------------------------------------------------------------------
lung: 656206 tasks: 900 meta: 300.0 325.0 325.0
603/603 [==============================] - 9s 16ms/sample
tota

348/348 [==============================] - 6s 16ms/sample
total: (3947, 6) lung: 211
----------------------------------------------------------------------------------------------------
lung: 688160 tasks: 800 meta: 265.0 331.0 331.0
510/510 [==============================] - 8s 16ms/sample
total: (3955, 6) lung: 212
----------------------------------------------------------------------------------------------------
lung: 648138 tasks: 729 meta: 290.0 320.0 320.0
512/512 [==============================] - 8s 16ms/sample
total: (3959, 6) lung: 213
----------------------------------------------------------------------------------------------------
lung: 684803 tasks: 800 meta: 280.0 350.0 350.0
563/563 [==============================] - 9s 16ms/sample
total: (3970, 6) lung: 214
----------------------------------------------------------------------------------------------------
lung: 664531 tasks: 900 meta: 300.0 350.0 350.0
544/544 [==============================] - 9s 16ms/sample
total:

In [14]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))



results: (4172, 6) distinct lung: 222


In [15]:
filepath_1 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_1.csv'
filepath_5 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_5.csv'
if os.path.exists(filepath_1) and os.path.exists(filepath_5):
    df_lung_1 = pd.read_csv(filepath_or_buffer=filepath_1, index_col=['seriesuid'])
    df_lung_1.index = df_lung_1.index.astype('str')
    print('lung window 1:', df_lung_1.shape)
    
    df_lung_5 = pd.read_csv(filepath_or_buffer=filepath_5, index_col=['seriesuid'])
    df_lung_5.index = df_lung_5.index.astype('str')
    print('lung window 5:', df_lung_5.shape)
    
    df_lung = pd.concat([df_lung_1, df_lung_5])
    print('lung windows merged:', df_lung.shape)
    df_lung.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'.csv', encoding='utf-8')

lung window 1: (4172, 6)
lung window 5: (8852, 6)
lung windows merged: (13024, 6)
